# Semantic Search

In [69]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
import warnings
warnings.filterwarnings("ignore")

## Connecting to Elasticsearch

In [70]:
es = Elasticsearch("https://localhost:9200", basic_auth=("elastic", "Data_is_Lyfe"), verify_certs=False)

In [71]:
query_phrase = "medicines"

In [72]:
qr_vec = model.encode(query_phrase)

2 x Cosine(search_vec, title_vec) + 1 x Cosine(search_vec, desc_vec)

In [73]:
# SELECT tittle FROM DP_index SORT BY COSINE () 

In [74]:
qr = {
"script_score": {
# "min_score": 4.0,
"query": {"match_all": {}},
"script": {
"source": "2*(cosineSimilarity(params.query_vector, 'title_vec') + 1.0) + (cosineSimilarity(params.query_vector, 'description_vec') + 1.0)",
"params": {"query_vector": qr_vec}
}
}
}

In [75]:
qr_fuzz = {
"script_score": {
"query": {
    "fuzzy": {
      "title.keyword": {
        "value": "custumer"
      }
    }
},
"script": {
"source": "2*(cosineSimilarity(params.query_vector, 'title_vec') + 1.0) + (cosineSimilarity(params.query_vector, 'description_vec') + 1.0)",
"params": {"query_vector": qr_vec}
}
}
}

In [76]:
search_res = [{"title":t["_source"]["title"], "score":t["_score"]} for t in es.search(query=qr, index = "dp_test_1")["hits"]["hits"]]

In [77]:
search_res

[{'title': 'Healthcare_360', 'score': 3.7168052},
 {'title': 'Customer_360', 'score': 3.2733698},
 {'title': 'Banking_360', 'score': 3.116454}]

In [78]:
search_1 = es.search(index="dp_test_1", body={"query": qr})

In [79]:
test_phrase = "credit card"

In [80]:
query_vec = model.encode(test_phrase)

In [81]:
len(query_vec)

384

In [82]:
qr = {
"script_score": {
"query": {"match_all": {}},
"script": {
"source": "2*(cosineSimilarity(params.query_vector, 'title_vec') + 1.0) + (cosineSimilarity(params.query_vector, 'description_vec') + 1.0)",
"params": {"query_vector": query_vec}
}
}
}

In [83]:
word_res = [{"title":t["_source"]["title"], "score":t["_score"]} for t in es.search(query=qr, index = "dp_test_1")["hits"]["hits"]]

In [84]:
word_res

[{'title': 'Banking_360', 'score': 4.276055},
 {'title': 'Customer_360', 'score': 3.884015},
 {'title': 'Healthcare_360', 'score': 3.734417}]

# KNN search

In [105]:
test_phrase = "debt"

In [106]:
query_vec = model.encode(test_phrase)

In [107]:
qr ={
    "field": "title_vec",
    "query_vector": query_vec,
    "k": 3,
    "num_candidates": 10
  }

In [108]:
word_res = [{"title":t["_source"]["title"], "score":t["_score"]} for t in es.knn_search(knn = qr, index = "dp_test_1")["hits"]["hits"]]

In [109]:
word_res

[{'title': 'Banking_360', 'score': 0.4476},
 {'title': 'Healthcare_360', 'score': 0.41087684},
 {'title': 'Customer_360', 'score': 0.39695895}]